In [2]:
import sys; sys.path.append('../')
import os
import numpy

# Structs

Structs are containers for structured data, similar to NumPy arrays.
The key difference to arrays is, however, that the data contained in structs is structured, i.e. hierarchical and named.
This allows writing vectorized code for much more complex data structures than multi-dimensional arrays.

The `struct` API constitues the bases for the the [`math` API](NumPy_and_TensorFlow_Execution.md).

## Building structs from lists and dicts

The following cell declares three structs, `a`, `b` and `c`. A full list of what classes can be used to construct structs is given later.

In [3]:
from phi import struct
a = [1, numpy.zeros([2])]
b = {'x0': 0, 'x1': 0.5, 'x2': 1}
c = (a, b)

Like arrays and tensors, structs have data types and shapes. Since a struct can contain many tensors with different types and shapes, these properties return the results in the same structure as the original struct.

In [4]:
struct.dtype(a)

[int, numpy.ndarray]

In [5]:
struct.shape(a)

[(), ()]

Functions in `phi.math` can be called on structs. This broadcasts the corresponding calls to all contained arrays.

In [6]:
from phi import math
math.sin(c)

([0.8414709848078965, array([0., 0.])],
 {'x0': 0.0, 'x1': 0.479425538604203, 'x2': 0.8414709848078965})

Note that math.sin calls numpy.sin in this case.
If the struct contained TensorFlow or PyTorch tensors, the corresponding sin functions would be called instead.

## Types of structs

The following types are considered structs:
- Lists
- Tuples
- Dicts containing strings as keys
- NumPy arrays with `dtype=numpy.object`
- Subclasses of [`phi.struct.Struct`](../phi/struct/struct.py)

All `phi.math` functions and functions of the struct API work with any of the above types.

While all entries of lists, tuples, dicts and NumPy arrays are expected to hold data,
subclasses of the `Struct` class can define further properties which are not subject to the above mentioned functions.

Struct items are separated into three categories:
- *Variables*: properties that change over time. They span the state space of a physical system in which the system moves over time.
- *Constants*: system characteristics that specify a certain system but do not change over time.
- *Derived quantities*: properties that may change over time but can fully be derived from variables and constants.

Variables are expected to change when a struct is run through an algorithm while constants should generally only be changed by the user. Each item can hold data (tensors), other structs or other values.

Typically, variables hold data while constants hold scalar values, booleans or strings.

## Iterating over structs

The struct interface provides the function `map` which iterates over all data-holding items of a struct and its sub-structs by default.

All items of lists and dicts are considered to be data-holding variables.

In [7]:
struct.map(lambda x: str(x), c)

(['1', '[0. 0.]'], {'x0': '0', 'x1': '0.5', 'x2': '1'})

If we only wanted to affect the tensors directly held by `data`, we could call

In [8]:
struct.map(lambda x: str(x), c, recursive=False)

('[1, array([0., 0.])]', "{'x0': 0, 'x1': 0.5, 'x2': 1}")

Assume, we want only dicts to be stringified as a whole.
This can be achieved by defining a leaf_condition.

In [9]:
struct.map(lambda x: str(x), c, leaf_condition=lambda x: isinstance(x, dict))

(['1', '[0. 0.]'], "{'x0': 0, 'x1': 0.5, 'x2': 1}")

The parameter `item_condition` can further specify which types of items should be affected by a struct operation.
The constants `VARIABLES`, `CONSTANTS`, `DATA` and `ALL_ITEMS` are part of the `struct` package and can be used for the `item_condition`.

The item condition can also be set through the context. Operations within a `with item_condition:` block that do not override the `item_condition` parameter, use the context item condition instead. The default context item condition is `DATA`.

In some cases we require additional information when mapping a struct; not just the value but also where it is stored.
When calling `map(.., trace=True)`, a [`Trace`](../phi/struct/functions.py) object is passed to the mapping function `f` instead of the value. In addition to retrieving the value via `trace.value`, it provides access to the attribute key as `trace.key` and the parent structs via `trace.parent`.

In [10]:
struct.map(lambda trace: trace.key, c, trace=True)

([0, 1], {'x0': 'x0', 'x1': 'x1', 'x2': 'x2'})

## Usages in Φ<sub>*Flow*</sub>

In Φ<sub>*Flow*</sub>, structs are used to represent simulation states.
Not only the state base class -- [`phi.physics.physics.State`](../phi/physics/physics.py) -- extends `Struct`, but also objects depended upon by states such as `Domain`, `Sphere`, `CenteredGrid` etc. inherit from `Struct`.

Variables hold the current state of the system (e.g. current velocity field) while constants describe the system itself (e.g. fluid viscosity).

Let's have a look at the structure of a `Fluid` state.

In [4]:
from phi.flow import Fluid, Domain
from phi.struct import names, flatten

fluid = Fluid(Domain([80, 64]))

with struct.VARIABLES:
    print("Variables: ", flatten(names(fluid)))

with struct.CONSTANTS:
    print("\nConstants: ", flatten(names(fluid)))

Variables:  ['age', 'density.age', 'density.data', 'velocity.age', 'velocity.data.0.age', 'velocity.data.0.data', 'velocity.data.1.age', 'velocity.data.1.data']

Constants:  ['buoyancy_factor', 'domain.boundaries.friction', 'domain.boundaries.name', 'domain.boundaries.periodic', 'domain.boundaries.solid', 'domain.resolution', 'domain.box.lower', 'domain.box.upper', 'name']


[Staggered grids](Staggered_Grids.md), as in `fluid.velocity`, are vector fields where the arrays for each component have different shapes.

## Tensor initialization

Initializer functions such as `zeros` or `placeholder` internally call their counterparts in NumPy or TensorFlow.
They can take 1D-tensors describing the shape as input but also support structs holding shapes.

Some states simplify this even further by allowing a syntax like `Fluid(density=zeros)` or `Fluid(velocity=placeholder)`.

In [26]:
math.zeros({'small array': (1,1), 'large array': [1, 64, 64]})

{'small array': array([[0.]], dtype=float32),
 'large array': array([[[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]]], dtype=float32)}

### Data I/O

The data writing and reading system accepts structs and automatically infers their names from the attributes.
See the [data documentation](Reading_and_Writing_Data.md).

### Session

The TensorFlow [`Session`](../phi/tf/session.py) class is a customized version of `tf.Session` which accepts structs for the `fetches` argument as well as inside the `feed_dict`.

This can be used to quickly run states through a graph like so:

In [27]:
from phi.tf.flow import *
numpy_state = Fluid(Domain([16, 16]), density=math.zeros, velocity=math.zeros)
placeholder_state = Fluid(Domain([16, 16]), density=placeholder, velocity=placeholder)
output_state = INCOMPRESSIBLE_FLOW.step(placeholder_state)
session = Session(None)
numpy_state = session.run(output_state, {placeholder_state: numpy_state})
numpy_state

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning:

Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.




For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Colocations handled automatically by placer.


Fluid[density: Grid[16x16(1), size=[16. 16.]], velocity: StaggeredGrid[16x16, size=[16. 16.]]]

## Validity

As structs are supposed to hold data in a specific structure, there is a preferred data type for each entry.
For a `CenteredGrid`, the `data` attribute should be a tensor or array with a certain rank and the `velocity` of a `Fluid` object should be a `StaggeredGrid`.

An item is _valid_ if its value fulfills all those restrictions and can be passed to a solver.
Subclasses of `Struct` implement validity checks and may modify their entries to make them valid.
This allows the shorthand notation `Fluid(density=1)` to create a `CenteredGrid` full of ones.

When representing some other property such as the `shape` or `dtype` of a struct, these restrictions do not apply.
To model this behavior properly, `Struct` objects remember their content type.
Item validation is only performed if the content type is equal to `VALID`.

# Immutability
While structs can be mutable in principle, the public struct API does not allow for changing a struct.
Variables and constants can be "changed" using the `copy_with` function and `copied_with` method.
This does not alter the struct but creates a duplicate with the new values.

## Implementing a custom struct

Variables and constants are declared using the decorators `struct.variable()` and `struct.constant()`, respectively.
Variables are marked as data-holding by default, constants as non-data.
These items create read-only attributes which should be changed only using the inherited `copied_with()` method.

The `generate` method can be used to quickly generate the Python skeleton of a custom struct.

In [42]:
from phi.struct.python_generator import generate
print(generate('MyStruct', variables=['v'], constants=['c'], others=['other']))


from phi import struct


@struct.definition()
class MyStruct(struct.Struct):
    
    def __init__(self, v, c, other, **kwargs):
        struct.Struct.__init__(self, **struct.kwargs(locals(), ignore=['other']))
        self._other = other
    
    @struct.variable()
    def v(self, v):
        return v

    @struct.constant()
    def c(self, c):
        return c
        
    @property
    def other(self):
        return self._other
        



The methods themselves are used for validation. In addition to `self`, each attribute and property gets the intended value as an input. The function can either directly return this value without any validity checks, raise an error for invalid values or transform the value into a valid value.

In [43]:
from phi import struct


@struct.definition()
class MyStruct(struct.Struct):
    
    def __init__(self, v, c, other, **kwargs):
        struct.Struct.__init__(self, **struct.kwargs(locals(), ignore=['other']))
        self._other = other
    
    @struct.variable()
    def v(self, v):
        return v

    @struct.constant()
    def c(self, c):
        return c
        
    @property
    def other(self):
        return self._other

We can iterate over specific items the same way as before.

In [45]:
mystruct = MyStruct(v=0, c=0, other=None)

print('--- Variables ---')
with struct.VARIABLES:
    struct.map(print_name, mystruct, trace=True, content_type=struct.INVALID);

print('\n--- Constants ---')
with struct.CONSTANTS:
    struct.map(print_name, mystruct, trace=True, content_type=struct.INVALID);

--- Variables ---
v  (int)

--- Constants ---
c  (int)


By inheriting from `Struct`, `MyStruct` obtains implementations for `dtype` and `shape`, making it look like a tensor. 

In [52]:
print(mystruct.dtype)

MyStruct[<function dtype at 0x000002CE5BCBA378>]


In [53]:
print(mystruct.dtype.v, mystruct.dtype.c)

int32 0


In [54]:
print(mystruct.shape.v, mystruct.shape.c)

() 0


Note that `shape` and `dtype` use the context item condition (`DATA` by default). Therefore only the variable `v` is affected at the moment. To obtain the shapes of other items, we can use the `with item_condition:` syntax.

In [55]:
with struct.ALL_ITEMS:
    print(mystruct.dtype.v, mystruct.dtype.c)

int32 int32


The inherited `copied_with` method can be used to "change" variables and constants.

In [64]:
changed = mystruct.copied_with(v=1, c=1)
print(changed.v, changed.c)

1 1


## Custom validation functions
Each item is declared as a function. This function is called upon validation to return a valid value for the item or raise an error.

Let's have the variable always be of type float and convert the constant to a string.

In [65]:
@struct.definition()
class MyStruct(struct.Struct):
    
    def __init__(self, v, c, other, **kwargs):
        struct.Struct.__init__(self, **struct.kwargs(locals(), ignore=['other']))
        self._other = other
    
    @struct.variable()
    def v(self, v):
        return float(v)

    @struct.constant()
    def c(self, c):
        return str(c)
        
    @property
    def other(self):
        return self._other

In [67]:
mystruct = MyStruct(v=0, c=0, other=None)
with struct.ALL_ITEMS:
    print(mystruct.dtype.v, mystruct.dtype.c)

float64 <class 'str'>


To skip validation, we could declare a different content type.

In [71]:
mystruct = MyStruct(v=0, c=0, other=None, content_type=struct.INVALID)
with struct.ALL_ITEMS:
    print(mystruct.dtype.v, mystruct.dtype.c)

int32 int32
